<a href="https://colab.research.google.com/github/panzershracker/Web-scraping/blob/master/func_tests2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import requests, re
from bs4 import BeautifulSoup as soup
from pprint import pprint



In [0]:
mask = '[а-я0-9—]+'
vacancy = 'программист'
language = 'python'
city = 'moskva.'
salary = '?st=1'
page = '&p=1'
sort = '?sort=date'

In [0]:
def get_vac_names():

    vac_name = soup(resp.text, 'lxml').find_all('a', {'data-category': "sourceAway"})

    vac_names = []

    for i in range(0, len(vac_name)):
        vac_names.append(vac_name[i].getText())

    return vac_names




In [0]:
def get_salaries():
   
    h2 = str(soup(resp.text, 'lxml').find_all('h2'))
    
    b = list(soup(h2, 'lxml').find_all('b'))
    
    salaries = []
    
    for i in b:
      sal = re.findall(mask, str(i))
      salaries.append(''.join(sal))
      
    return salaries



In [0]:
def get_links():
  
  vac_link = soup(resp.text, 'lxml').find_all('a', {'data-category': "sourceAway"})

  vac_links = []

  for i in range(0, len(vac_link)):
    vac_links.append(vac_link[i].get('href'))
        

  return vac_links


In [0]:
def salary_separator():
  
  ot = []
  do = []
  
  salaries = get_salaries()
  
  for i in salaries:
    
    if i.startswith('от'):
      
      ot.append(int(i[2:]))
      do.append(0)
    
    elif i.startswith('до'):
      
      if i.startswith('дог'):
        
        ot.append(0)
        do.append(0)
      
      else:
        
        do.append(int(i[2:]))
        ot.append(0)
      
    else:
      
      a = i.split('—')
      
      ot.append(int(a[0]))
      do.append(int(a[1]))
      
  return ot, do

In [0]:
# c = '1000—2000'

# a, b = c.split('—')

# print(a, b)

In [0]:
with requests.Session() as session:

    session.headers['User-Agent'] = 'Mozilla/5.0 (X11; Linux x86_64; rv:63.0) Gecko/20100101 Firefox/63.0'

    url = f'https://{city}gorodrabot.ru/{vacancy}_{language}{sort}{salary}{page}'
    resp = session.get(url)

    print(f'gorodrabot.ru status code is: {resp.status_code}')

    df = pd.DataFrame({'name': get_vac_names(),
                       'salary min': salary_separator()[0],
                       'salary max': salary_separator()[1],
                       'link': get_links()})

In [19]:
pprint(get_vac_names())

['Ведущий разработчик Python',
 'Junior Python developer / Начинающий разработчик Python',
 'Backend-разработчик (python 3)',
 'Backend разработчик на Python во внутреннее облако Яндек...',
 'Senior Fullstack разработчик',
 'Разработчик интерфейсов',
 'Разработчик RPA (UiPath)',
 'Программист 1С',
 'Программист-разработчик 1С',
 'Intelligent R&D Tools Technical Expert',
 'Веб-аналитик',
 'Руководитель по дистанционному обучению',
 'Билд-инженер/Программист Python (Continuous Integration)',
 'Разработчик Python',
 'Программист Python',
 'Разработчик Python',
 'Программист 1С',
 'Web-программист, Web-разработчик']


In [20]:
pprint(get_salaries())

['договорная',
 '60000—70000',
 '100000—200000',
 'договорная',
 'от150000',
 'договорная',
 'договорная',
 'договорная',
 'от80000',
 'договорная',
 'договорная',
 'договорная',
 'договорная',
 'от120000',
 'договорная',
 'от300000',
 'договорная',
 'договорная',
 'договорная',
 'договорная',
 'договорная',
 '160000—200000',
 'договорная',
 '100000—180000',
 'договорная',
 'договорная']


In [21]:
print(len(get_vac_names()))
print(len(get_salaries()))
print(len(get_links()))
print(len(salary_separator()[0]))
print(len(salary_separator()[1]))

18
26
18
26
26
